# Data Loader (Cue Conflict)

In [58]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

dataset_path = '/kaggle/input/cue-conflict/'

transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor(),        
    # transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]) 
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

])

dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=2)
print(f"Class names: {dataset.classes}")

for images, labels in data_loader:
    print(f"Batch of images: {images.shape}")
    print(f"Batch of labels: {labels}")
    break

Class names: ['airplane', 'bear', 'bicycle', 'bird', 'boat', 'bottle', 'car', 'cat', 'chair', 'clock', 'dog', 'elephant', 'keyboard', 'knife', 'oven', 'truck']
Batch of images: torch.Size([32, 3, 224, 224])
Batch of labels: tensor([14,  6, 15, 14, 13,  9, 13, 13,  0, 10, 10,  7, 13, 12, 12, 15, 11,  3,
         3, 14,  3, 14,  9,  3,  1,  2,  0, 10,  1,  2,  1,  1])


# Teacher Models

**VIT**

In [2]:
import torch
from torch import nn
from torchvision.models import vit_b_16

class ViTModel(nn.Module): 
    def __init__(self, num_classes, pretrained=True):
        super(ViTModel, self).__init__()
        self.model = vit_b_16(pretrained=pretrained)
        
        for param in self.model.parameters():
            param.requires_grad = False
            
        in_features = self.model.heads.head.in_features
        self.model.heads.head = nn.Linear(in_features, num_classes)
        for param in self.model.heads.head.parameters():
            param.requires_grad = True

    def forward(self, x):
        return self.model(x)

    def get_model(self):
        return self.model

# num_classes = 10 
# vit_model = ViTModel(num_classes=num_classes, pretrained=True)
# model_instance = vit_model.get_model()
# print(model_instance)

**ResNet50**

In [63]:
# import torch
# from torchvision import models
# import torch.nn as nn

# class ResNet50Model(nn.Module):
#     def __init__(self, num_classes, pretrained=True):
#         super(ResNet50Model, self).__init__()
#         self.model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT if pretrained else None)
    
#         for param in self.model.parameters():
#             param.requires_grad = False
#         in_features = self.model.fc.in_features
#         self.model.fc = nn.Linear(in_features, num_classes)
#         for param in self.model.fc.parameters():
#             param.requires_grad = True

#     def forward(self, x):
#         return self.model(x)
#     def get_model(self):
#         return self.model

import torch
from torchvision import models
import torch.nn as nn

class ResNet50Model(nn.Module):
    def __init__(self, num_classes, pretrained=True):
        super(ResNet50Model, self).__init__()
        if pretrained:
            # Load the model with pretrained weights
            self.model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        else:
            # Load the model without pretrained weights
            self.model = models.resnet50(weights=None)

        # Freeze all parameters except the final fully connected layer
        for param in self.model.parameters():
            param.requires_grad = False
        
        # Modify the final fully connected layer to match the number of classes
        in_features = self.model.fc.in_features
        self.model.fc = nn.Linear(in_features, num_classes)
        
        # Unfreeze the parameters of the final fully connected layer
        for param in self.model.fc.parameters():
            param.requires_grad = True

    def forward(self, x):
        return self.model(x)

    def get_model(self):
        return self.model


# num_classes = 10 
# resnet_model = ResNet50Model(num_classes=num_classes, pretrained=True)
# model_instance = resnet_model.get_model()
# print(model_instance)


**DenseNet121**

In [4]:
import torch
from torchvision import models
import torch.nn as nn

class DenseNet121Model(nn.Module):
    def __init__(self, num_classes, pretrained=True):
        super(DenseNet121Model, self).__init__()
        self.model = models.densenet121(pretrained=pretrained)

        for param in self.model.features.parameters():
            param.requires_grad = False
        in_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(in_features, num_classes)
        for param in self.model.classifier.parameters():
            param.requires_grad = True

    def forward(self, x):
        return self.model(x)
    def get_model(self):
        return self.model

# num_classes = 10  
# densenet_model = DenseNet121Model(num_classes=num_classes, pretrained=True)
# model_instance = densenet_model.get_model()
# print(model_instance)

**Vgg-16**

In [27]:
import torch
import torchvision.models as models
import torch.nn as nn

class VGG16Model(nn.Module):
    def __init__(self, num_classes, pretrained=True):
        super(VGG16Model, self).__init__()
        self.model = models.vgg16(pretrained=pretrained)

        for param in self.model.features.parameters():
            param.requires_grad = False
        in_features = self.model.classifier[6].in_features
        self.model.classifier[6] = nn.Linear(in_features, num_classes)
        for param in self.model.classifier[6].parameters():
            param.requires_grad = True

    def forward(self, x):
        return self.model(x)
        
    def get_model(self):
        return self.model
        
# num_classes = 10  
# vgg_model = VGG16Model(num_classes=num_classes, pretrained=True)
# model_instance = vgg_model.get_model()
# print(model_instance)

In [6]:
# num_classes = 16

# vgg16_model = VGG16Model(num_classes=num_classes, pretrained=True).get_model()
# print('VGG16 model loaded')
# resnet50_model = ResNet50Model(num_classes=num_classes, pretrained=True).get_model()
# print('ResNet50 model loaded')
# vit_model = ViTModel(num_classes=num_classes, pretrained=True).get_model()
# print('Vision Transformer (ViT) model loaded')
# densenet121_model = DenseNet121Model(num_classes=num_classes, pretrained=True).get_model()
# print('DenseNet121 model loaded')

# Student Model

In [7]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm
from torchvision.models import vgg11
from torchvision import models

class VGG11(nn.Module):
    def __init__(self, num_classes=16, pretrained=True):
        super(VGG11, self).__init__()

        self.vgg11 = models.vgg11(pretrained=pretrained)
        for param in self.vgg11.features.parameters():
            param.requires_grad = False
            
        self.vgg11.classifier[6] = nn.Linear(self.vgg11.classifier[6].in_features, num_classes)
    
    def forward(self, x):
        return self.vgg11(x)

    def get_model(self):
        return self.vgg11

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vgg11_model = VGG11(num_classes=16, pretrained=True)
vgg11_model = vgg11_model.to(device)
print('VGG11 model loaded')


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG11_Weights.IMAGENET1K_V1`. You can also use `weights=VGG11_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg11-8a719046.pth" to /root/.cache/torch/hub/checkpoints/vgg11-8a719046.pth
100%|██████████| 507M/507M [00:02<00:00, 217MB/s] 


VGG11 model loaded


# Dataloader

In [9]:
from torch.utils.data import DataLoader, random_split

def create_data_loaders(dataset_path, batch_size, val_split=0.2, seed=42):
    torch.manual_seed(seed)

    transform = transforms.Compose([
        transforms.Resize((224, 224)), 
        transforms.ToTensor(),        
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])
    
    dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
    val_size = int(len(dataset) * val_split)
    train_size = len(dataset) - val_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
    
    print(f"Train size: {train_size}, Validation size: {val_size}")
    print(f"Class names: {dataset.classes}")
    
    return train_loader, val_loader

# FineTuning

In [10]:
import torch
import torch.optim as optim
import torch.nn as nn
import os
import zipfile
from tqdm import tqdm

def fine_tune_model(model_class, dataset_path, output_path, num_classes, epochs=10, batch_size=32, learning_rate=0.001):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    train_loader, val_loader = create_data_loaders(dataset_path, batch_size)
    model = model_class(num_classes=num_classes, pretrained=True)
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()

    # Optimizer configuration for ViT and ResNet
    if isinstance(model, ViTModel):  # For ViT
        optimizer = optim.Adam(model.get_model().heads.head.parameters(), lr=learning_rate)
    elif isinstance(model, ResNet50Model):  # For ResNet
        optimizer = optim.Adam(model.get_model().fc.parameters(), lr=learning_rate)  # Use 'fc' for ResNet
    else:  # For other models (e.g., DenseNet, etc.)
        optimizer = optim.Adam(model.get_model().classifier.parameters(), lr=learning_rate)

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        train_correct = 0
        total_samples = 0

        print(f"Epoch {epoch+1}/{epochs}: Training started...")
        for images, labels in tqdm(train_loader, desc="Training", unit="batch"):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            train_correct += torch.sum(preds == labels)
            total_samples += images.size(0)

        train_loss_avg = train_loss / total_samples
        train_accuracy = (train_correct.double() / total_samples) * 100
        model.eval()
        val_loss = 0.0
        val_correct = 0
        with torch.no_grad():
            print("Validation started...")
            for images, labels in tqdm(val_loader, desc="Validating", unit="batch"):
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * images.size(0)
                _, preds = torch.max(outputs, 1)
                val_correct += torch.sum(preds == labels)

        val_loss_avg = val_loss / len(val_loader.dataset)
        val_accuracy = (val_correct.double() / len(val_loader.dataset)) * 100
        print(f"Epoch {epoch+1}/{epochs}: "
              f"Training Accuracy: {train_accuracy:.4f}, "
              f"Training Loss: {train_loss_avg:.4f} | "
              f"Validation Accuracy: {val_accuracy:.4f}, "
              f"Validation Loss: {val_loss_avg:.4f}")

    model_name = model_class.__name__.lower()
    model_path = os.path.join(output_path, f"{model_name}_fine_tuned_{num_classes}_classes.pth")
    torch.save(model.state_dict(), model_path)

    # torch.save(model.get_model().state_dict(), model_path)
    print(f"Model saved at {model_path}")
    
    zip_path = model_path.replace(".pth", ".zip")
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        zipf.write(model_path, os.path.basename(model_path))
    # os.remove(model_path)
    print(f"Model saved and zipped to {zip_path}")


Fine tuning all models on Labelled Image Net

In [11]:
dataset_path = "/kaggle/input/labelledimagenet"  
output_path = "/kaggle/working/FinetunedModels" 
os.makedirs(output_path, exist_ok=True)
model_class = VGG11
fine_tune_model(model_class, dataset_path, output_path, num_classes=16, epochs=10, batch_size=128, learning_rate=1e-4)

Train size: 6029, Validation size: 1507
Class names: ['airplane', 'bear', 'bicycle', 'bird', 'boat', 'bottle', 'car', 'cat', 'chair', 'clock', 'dog', 'elephant', 'keyboard', 'knife', 'oven', 'truck']
Epoch 1/10: Training started...


Training: 100%|██████████| 48/48 [00:34<00:00,  1.39batch/s]


Validation started...


Validating: 100%|██████████| 12/12 [00:08<00:00,  1.35batch/s]


Epoch 1/10: Training Accuracy: 82.9325, Training Loss: 0.6153 | Validation Accuracy: 93.5634, Validation Loss: 0.2122
Epoch 2/10: Training started...


...

Epoch 9/10: Training Accuracy: 99.9834, Training Loss: 0.0017 | Validation Accuracy: 95.0232, Validation Loss: 0.2078
Epoch 10/10: Training started...


Training: 100%|██████████| 48/48 [00:21<00:00,  2.20batch/s]


Validation started...


Validating: 100%|██████████| 12/12 [00:06<00:00,  1.93batch/s]


Epoch 10/10: Training Accuracy: 100.0000, Training Loss: 0.0010 | Validation Accuracy: 95.0232, Validation Loss: 0.1961
Model saved at /kaggle/working/FinetunedModels/vgg11_fine_tuned_16_classes.pth
Model saved and zipped to /kaggle/working/FinetunedModels/vgg11_fine_tuned_16_classes.zip


In [12]:
dataset_path = "/kaggle/input/labelledimagenet" 
output_path = "/kaggle/working/FinetunedModels"
os.makedirs(output_path, exist_ok=True)
model_class = ViTModel 
fine_tune_model(model_class, dataset_path, output_path, num_classes=16, epochs=10, batch_size=128, learning_rate=1e-4)

Train size: 6029, Validation size: 1507
Class names: ['airplane', 'bear', 'bicycle', 'bird', 'boat', 'bottle', 'car', 'cat', 'chair', 'clock', 'dog', 'elephant', 'keyboard', 'knife', 'oven', 'truck']


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth
100%|██████████| 330M/330M [00:01<00:00, 201MB/s] 


Epoch 1/10: Training started...


Training: 100%|██████████| 48/48 [00:38<00:00,  1.26batch/s]


Validation started...


...

Epoch 9/10: Training Accuracy: 88.6051, Training Loss: 0.9658 | Validation Accuracy: 87.7240, Validation Loss: 0.9234
Epoch 10/10: Training started...


Training: 100%|██████████| 48/48 [00:38<00:00,  1.25batch/s]


Validation started...


Validating: 100%|██████████| 12/12 [00:10<00:00,  1.18batch/s]


Epoch 10/10: Training Accuracy: 89.7495, Training Loss: 0.8748 | Validation Accuracy: 88.5202, Validation Loss: 0.8415
Model saved at /kaggle/working/FinetunedModels/vitmodel_fine_tuned_16_classes.pth
Model saved and zipped to /kaggle/working/FinetunedModels/vitmodel_fine_tuned_16_classes.zip


In [13]:
dataset_path = "/kaggle/input/labelledimagenet" 
output_path = "/kaggle/working/FinetunedModels"  
os.makedirs(output_path, exist_ok=True)
model_class = VGG16Model  
fine_tune_model(model_class, dataset_path, output_path, num_classes=16, epochs=10, batch_size=128, learning_rate=1e-4)

Train size: 6029, Validation size: 1507
Class names: ['airplane', 'bear', 'bicycle', 'bird', 'boat', 'bottle', 'car', 'cat', 'chair', 'clock', 'dog', 'elephant', 'keyboard', 'knife', 'oven', 'truck']


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 221MB/s] 


Epoch 1/10: Training started...


Training: 100%|██████████| 48/48 [00:23<00:00,  2.03batch/s]


Validation started...


...

Epoch 9/10: Training Accuracy: 100.0000, Training Loss: 0.0010 | Validation Accuracy: 96.5494, Validation Loss: 0.1415
Epoch 10/10: Training started...


Training: 100%|██████████| 48/48 [00:23<00:00,  2.03batch/s]


Validation started...


Validating: 100%|██████████| 12/12 [00:06<00:00,  1.79batch/s]


Epoch 10/10: Training Accuracy: 100.0000, Training Loss: 0.0005 | Validation Accuracy: 96.6821, Validation Loss: 0.1406
Model saved at /kaggle/working/FinetunedModels/vgg16model_fine_tuned_16_classes.pth
Model saved and zipped to /kaggle/working/FinetunedModels/vgg16model_fine_tuned_16_classes.zip


In [14]:
dataset_path = "/kaggle/input/labelledimagenet"  
output_path = "/kaggle/working/FinetunedModels" 
os.makedirs(output_path, exist_ok=True)
model_class = DenseNet121Model  
fine_tune_model(model_class, dataset_path, output_path, num_classes=16, epochs=10, batch_size=128, learning_rate=1e-4)

Train size: 6029, Validation size: 1507
Class names: ['airplane', 'bear', 'bicycle', 'bird', 'boat', 'bottle', 'car', 'cat', 'chair', 'clock', 'dog', 'elephant', 'keyboard', 'knife', 'oven', 'truck']


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 157MB/s]


Epoch 1/10: Training started...


Training: 100%|██████████| 48/48 [00:22<00:00,  2.11batch/s]


Validation started...


...

Epoch 9/10: Training Accuracy: 86.5981, Training Loss: 0.5295 | Validation Accuracy: 87.0604, Validation Loss: 0.5057
Epoch 10/10: Training started...


Training: 100%|██████████| 48/48 [00:22<00:00,  2.15batch/s]


Validation started...


Validating: 100%|██████████| 12/12 [00:06<00:00,  1.85batch/s]


Epoch 10/10: Training Accuracy: 88.3563, Training Loss: 0.4741 | Validation Accuracy: 88.7857, Validation Loss: 0.4600
Model saved at /kaggle/working/FinetunedModels/densenet121model_fine_tuned_16_classes.pth
Model saved and zipped to /kaggle/working/FinetunedModels/densenet121model_fine_tuned_16_classes.zip


In [15]:
dataset_path = "/kaggle/input/labelledimagenet"  
output_path = "/kaggle/working/FinetunedModels" 
os.makedirs(output_path, exist_ok=True)
model_class = ResNet50Model
fine_tune_model(model_class, dataset_path, output_path, num_classes=16, epochs=10, batch_size=128, learning_rate=1e-4)

Train size: 6029, Validation size: 1507
Class names: ['airplane', 'bear', 'bicycle', 'bird', 'boat', 'bottle', 'car', 'cat', 'chair', 'clock', 'dog', 'elephant', 'keyboard', 'knife', 'oven', 'truck']


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 200MB/s]


Epoch 1/10: Training started...


Training: 100%|██████████| 48/48 [00:22<00:00,  2.16batch/s]


Validation started...


...

Epoch 9/10: Training Accuracy: 84.1101, Training Loss: 0.7245 | Validation Accuracy: 85.6669, Validation Loss: 0.7457
Epoch 10/10: Training started...


Training: 100%|██████████| 48/48 [00:21<00:00,  2.23batch/s]


Validation started...


Validating: 100%|██████████| 12/12 [00:06<00:00,  1.97batch/s]


Epoch 10/10: Training Accuracy: 85.7356, Training Loss: 0.6709 | Validation Accuracy: 85.9987, Validation Loss: 0.6895
Model saved at /kaggle/working/FinetunedModels/resnet50model_fine_tuned_16_classes.pth
Model saved and zipped to /kaggle/working/FinetunedModels/resnet50model_fine_tuned_16_classes.zip


**FineTuning on Cue Conflict**

In [52]:
dataset_path = "/kaggle/input/cue-conflict"  
output_path = "/kaggle/working/FinetunedModelsCC" 
os.makedirs(output_path, exist_ok=True)
model_class = VGG11
fine_tune_model(model_class, dataset_path, output_path, num_classes=16, epochs=10, batch_size=128, learning_rate=1e-4)

Train size: 1024, Validation size: 256
Class names: ['airplane', 'bear', 'bicycle', 'bird', 'boat', 'bottle', 'car', 'cat', 'chair', 'clock', 'dog', 'elephant', 'keyboard', 'knife', 'oven', 'truck']
Epoch 1/10: Training started...


Training: 100%|██████████| 8/8 [00:03<00:00,  2.54batch/s]


Validation started...


Validating: 100%|██████████| 2/2 [00:01<00:00,  1.61batch/s]


Epoch 1/10: Training Accuracy: 8.2031, Training Loss: 2.7956 | Validation Accuracy: 21.8750, Validation Loss: 2.6138
Epoch 2/10: Training started...


...

Epoch 9/10: Training Accuracy: 98.0469, Training Loss: 0.1161 | Validation Accuracy: 69.5312, Validation Loss: 1.0241
Epoch 10/10: Training started...


Training: 100%|██████████| 8/8 [00:03<00:00,  2.51batch/s]


Validation started...


Validating: 100%|██████████| 2/2 [00:01<00:00,  1.67batch/s]


Epoch 10/10: Training Accuracy: 99.6094, Training Loss: 0.0663 | Validation Accuracy: 69.9219, Validation Loss: 1.0089
Model saved at /kaggle/working/FinetunedModelsCC/vgg11_fine_tuned_16_classes.pth
Model saved and zipped to /kaggle/working/FinetunedModelsCC/vgg11_fine_tuned_16_classes.zip


In [53]:
dataset_path = "/kaggle/input/cue-conflict"  
output_path = "/kaggle/working/FinetunedModelsDistillation" 
os.makedirs(output_path, exist_ok=True)
model_class = VGG11
fine_tune_model(model_class, dataset_path, output_path, num_classes=16, epochs=2, batch_size=128, learning_rate=1e-4)

Train size: 1024, Validation size: 256
Class names: ['airplane', 'bear', 'bicycle', 'bird', 'boat', 'bottle', 'car', 'cat', 'chair', 'clock', 'dog', 'elephant', 'keyboard', 'knife', 'oven', 'truck']
Epoch 1/2: Training started...


Training: 100%|██████████| 8/8 [00:03<00:00,  2.46batch/s]


Validation started...


Validating: 100%|██████████| 2/2 [00:01<00:00,  1.65batch/s]


Epoch 1/2: Training Accuracy: 8.2031, Training Loss: 2.7956 | Validation Accuracy: 21.8750, Validation Loss: 2.6138
Epoch 2/2: Training started...


Training: 100%|██████████| 8/8 [00:03<00:00,  2.47batch/s]


Validation started...


Validating: 100%|██████████| 2/2 [00:01<00:00,  1.62batch/s]


Epoch 2/2: Training Accuracy: 37.4023, Training Loss: 2.2814 | Validation Accuracy: 34.3750, Validation Loss: 2.3377
Model saved at /kaggle/working/FinetunedModelsDistillation/vgg11_fine_tuned_16_classes.pth
Model saved and zipped to /kaggle/working/FinetunedModelsDistillation/vgg11_fine_tuned_16_classes.zip


In [60]:
dataset_path = "/kaggle/input/cue-conflict" 
output_path = "/kaggle/working/FinetunedModelsCC"
os.makedirs(output_path, exist_ok=True)
model_class = ViTModel 
fine_tune_model(model_class, dataset_path, output_path, num_classes=16, epochs=10, batch_size=128, learning_rate=1e-4)

Train size: 1024, Validation size: 256
Class names: ['airplane', 'bear', 'bicycle', 'bird', 'boat', 'bottle', 'car', 'cat', 'chair', 'clock', 'dog', 'elephant', 'keyboard', 'knife', 'oven', 'truck']
Epoch 1/10: Training started...


Training: 100%|██████████| 8/8 [00:07<00:00,  1.13batch/s]


Validation started...


Validating: 100%|██████████| 2/2 [00:02<00:00,  1.17s/batch]


Epoch 1/10: Training Accuracy: 4.4922, Training Loss: 2.8732 | Validation Accuracy: 4.6875, Validation Loss: 2.8530
Epoch 2/10: Training started...


...

Epoch 9/10: Training Accuracy: 24.3164, Training Loss: 2.5249 | Validation Accuracy: 21.0938, Validation Loss: 2.5798
Epoch 10/10: Training started...


Training: 100%|██████████| 8/8 [00:07<00:00,  1.10batch/s]


Validation started...


Validating: 100%|██████████| 2/2 [00:02<00:00,  1.19s/batch]


Epoch 10/10: Training Accuracy: 26.7578, Training Loss: 2.4862 | Validation Accuracy: 22.6562, Validation Loss: 2.5480
Model saved at /kaggle/working/FinetunedModelsCC/vitmodel_fine_tuned_16_classes.pth
Model saved and zipped to /kaggle/working/FinetunedModelsCC/vitmodel_fine_tuned_16_classes.zip


In [55]:
dataset_path = "/kaggle/input/cue-conflict" 
output_path = "/kaggle/working/FinetunedModelsCC"  
os.makedirs(output_path, exist_ok=True)
model_class = VGG16Model  
fine_tune_model(model_class, dataset_path, output_path, num_classes=16, epochs=10, batch_size=128, learning_rate=1e-4)

Train size: 1024, Validation size: 256
Class names: ['airplane', 'bear', 'bicycle', 'bird', 'boat', 'bottle', 'car', 'cat', 'chair', 'clock', 'dog', 'elephant', 'keyboard', 'knife', 'oven', 'truck']
Epoch 1/10: Training started...


Training: 100%|██████████| 8/8 [00:03<00:00,  2.18batch/s]


Validation started...


Validating: 100%|██████████| 2/2 [00:01<00:00,  1.37batch/s]


Epoch 1/10: Training Accuracy: 7.2266, Training Loss: 2.8259 | Validation Accuracy: 15.2344, Validation Loss: 2.6901
Epoch 2/10: Training started...


...

Epoch 9/10: Training Accuracy: 97.7539, Training Loss: 0.1379 | Validation Accuracy: 57.8125, Validation Loss: 1.3020
Epoch 10/10: Training started...


Training: 100%|██████████| 8/8 [00:03<00:00,  2.13batch/s]


Validation started...


Validating: 100%|██████████| 2/2 [00:01<00:00,  1.34batch/s]


Epoch 10/10: Training Accuracy: 98.0469, Training Loss: 0.0999 | Validation Accuracy: 57.8125, Validation Loss: 1.2765
Model saved at /kaggle/working/FinetunedModelsCC/vgg16model_fine_tuned_16_classes.pth
Model saved and zipped to /kaggle/working/FinetunedModelsCC/vgg16model_fine_tuned_16_classes.zip


In [56]:
dataset_path = "/kaggle/input/cue-conflict"  
output_path = "/kaggle/working/FinetunedModelsCC" 
os.makedirs(output_path, exist_ok=True)
model_class = DenseNet121Model  
fine_tune_model(model_class, dataset_path, output_path, num_classes=16, epochs=10, batch_size=128, learning_rate=1e-4)

Train size: 1024, Validation size: 256
Class names: ['airplane', 'bear', 'bicycle', 'bird', 'boat', 'bottle', 'car', 'cat', 'chair', 'clock', 'dog', 'elephant', 'keyboard', 'knife', 'oven', 'truck']
Epoch 1/10: Training started...


Training: 100%|██████████| 8/8 [00:03<00:00,  2.33batch/s]


Validation started...


Validating: 100%|██████████| 2/2 [00:01<00:00,  1.52batch/s]


Epoch 1/10: Training Accuracy: 8.3984, Training Loss: 2.8436 | Validation Accuracy: 8.9844, Validation Loss: 2.8101
Epoch 2/10: Training started...


...

Epoch 9/10: Training Accuracy: 19.6289, Training Loss: 2.6180 | Validation Accuracy: 13.6719, Validation Loss: 2.6751
Epoch 10/10: Training started...


Training: 100%|██████████| 8/8 [00:03<00:00,  2.24batch/s]


Validation started...


Validating: 100%|██████████| 2/2 [00:01<00:00,  1.47batch/s]


Epoch 10/10: Training Accuracy: 19.9219, Training Loss: 2.5978 | Validation Accuracy: 15.6250, Validation Loss: 2.6577
Model saved at /kaggle/working/FinetunedModelsCC/densenet121model_fine_tuned_16_classes.pth
Model saved and zipped to /kaggle/working/FinetunedModelsCC/densenet121model_fine_tuned_16_classes.zip


In [64]:
dataset_path = "/kaggle/input/cue-conflict"  
output_path = "/kaggle/working/FinetunedModelsCC" 
os.makedirs(output_path, exist_ok=True)
model_class = ResNet50Model
fine_tune_model(model_class, dataset_path, output_path, num_classes=16, epochs=10, batch_size=128, learning_rate=1e-4)

Train size: 1024, Validation size: 256
Class names: ['airplane', 'bear', 'bicycle', 'bird', 'boat', 'bottle', 'car', 'cat', 'chair', 'clock', 'dog', 'elephant', 'keyboard', 'knife', 'oven', 'truck']
Epoch 1/10: Training started...


Training: 100%|██████████| 8/8 [00:03<00:00,  2.39batch/s]


Validation started...


Validating: 100%|██████████| 2/2 [00:01<00:00,  1.57batch/s]


Epoch 1/10: Training Accuracy: 7.2266, Training Loss: 2.7822 | Validation Accuracy: 7.0312, Validation Loss: 2.7761
Epoch 2/10: Training started...


...

Epoch 9/10: Training Accuracy: 31.9336, Training Loss: 2.6020 | Validation Accuracy: 17.1875, Validation Loss: 2.6863
Epoch 10/10: Training started...


Training: 100%|██████████| 8/8 [00:03<00:00,  2.36batch/s]


Validation started...


Validating: 100%|██████████| 2/2 [00:01<00:00,  1.52batch/s]


Epoch 10/10: Training Accuracy: 34.8633, Training Loss: 2.5806 | Validation Accuracy: 20.3125, Validation Loss: 2.6766
Model saved at /kaggle/working/FinetunedModelsCC/resnet50model_fine_tuned_16_classes.pth
Model saved and zipped to /kaggle/working/FinetunedModelsCC/resnet50model_fine_tuned_16_classes.zip


# Evaluation

In [50]:
import torch
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from tqdm import tqdm

def load_fine_tuned_model(model_name, model_path, num_classes=16):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    if model_name == "VGG11":
        model = VGG11(num_classes=num_classes, pretrained=False)
    elif model_name == "VGG16":
        model = VGG16Model(num_classes=num_classes, pretrained=False)
    elif model_name == "ResNet50":
        model = ResNet50Model(num_classes=num_classes, pretrained=False)
    elif model_name == "DenseNet121":
        model = DenseNet121Model(num_classes=num_classes, pretrained=False)
    elif model_name == "ViT":
        model = ViTModel(num_classes=num_classes, pretrained=False)
    else:
        raise ValueError(f"Unsupported model name: {model_name}")

    model.load_state_dict(torch.load(model_path, map_location=device))
    model = model.to(device)
    model.eval()
    print(f'{model_name} model loaded and ready for evaluation.')
    return model

def evaluate_model(model, dataset_path, batch_size=32):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    transform = transforms.Compose([
        transforms.Resize((224, 224)), 
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])
    
    eval_dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
    eval_loader = DataLoader(eval_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
    
    print(f"Evaluating on dataset with {len(eval_dataset)} samples and classes: {eval_dataset.classes}")

    total_samples = 0
    correct_predictions = 0
    
    with torch.no_grad():
        for images, labels in tqdm(eval_loader, desc="Evaluating"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            
            correct_predictions += torch.sum(preds == labels).item()
            total_samples += labels.size(0)
    
    accuracy = (correct_predictions / total_samples) * 100
    print(f"Accuracy on the evaluation dataset: {accuracy:.4f} %")
    
    return accuracy

In [17]:
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
import re

def strip_label_suffix(label):
    return re.sub(r'\d+$', '', label)

def evaluate_cue_conflict_dataset(model, dataset_path, batch_size=32):

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])
    eval_dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
    eval_loader = DataLoader(eval_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    print(f"Evaluating on dataset with {len(eval_dataset)} samples and classes: {eval_dataset.classes}")

    shape_correct = 0
    texture_correct = 0
    total_samples = 0

    with torch.no_grad():
        for images, labels in tqdm(eval_loader, desc="Evaluating"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)

            for i, pred in enumerate(preds):
                filename = eval_dataset.samples[total_samples + i][0]
                shape_texture = os.path.basename(filename).split("-")
                shape_label = strip_label_suffix(shape_texture[0]) 
                texture_label = strip_label_suffix(shape_texture[1][:-4])  
                pred_class = eval_dataset.classes[pred]

                if pred_class == shape_label:
                    shape_correct += 1
                if pred_class == texture_label:
                    texture_correct += 1
            total_samples += len(preds)

    shape_accuracy = shape_correct / total_samples if total_samples > 0 else 0
    texture_accuracy = texture_correct / total_samples if total_samples > 0 else 0
    cue_accuracy = shape_accuracy + texture_accuracy
    shape_bias = shape_accuracy / cue_accuracy if cue_accuracy > 0 else 0
    texture_bias = 1 - shape_bias

    print("\nFinal Results:")
    print(f"  Shape Accuracy: {shape_accuracy:.4f}")
    print(f"  Texture Accuracy: {texture_accuracy:.4f}")
    print(f"  Cue Accuracy: {cue_accuracy:.4f}")
    print(f"  Shape Bias: {shape_bias:.4f}")
    print(f"  Texture Bias: {texture_bias:.4f}")

    # return {
    #     "Shape Accuracy": shape_accuracy,
    #     "Texture Accuracy": texture_accuracy,
    #     "Cue Accuracy": cue_accuracy,
    #     "Shape Bias": shape_bias,
    #     "Texture Bias": texture_bias
    # }

In [65]:
model_name = "VGG11"  
model_path = "/kaggle/working/FinetunedModels/vgg11_fine_tuned_16_classes.pth" 
eval_dataset_path1 = "/kaggle/input/labelledimagenet" 
eval_dataset_path2 = "/kaggle/input/cue-conflict" 
model = load_fine_tuned_model(model_name=model_name, model_path=model_path, num_classes=16)
evaluate_model(model, dataset_path=eval_dataset_path1, batch_size=32)
evaluate_cue_conflict_dataset(model, eval_dataset_path2, batch_size=32)

<ipython-input-50-1ecbffe9c678>:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=device))


VGG11 model loaded and ready for evaluation.
Evaluating on dataset with 7536 samples and classes: ['airplane', 'bear', 'bicycle', 'bird', 'boat', 'bottle', 'car', 'cat', 'chair', 'clock', 'dog', 'elephant', 'keyboard', 'knife', 'oven', 'truck']


Evaluating: 100%|██████████| 236/236 [00:27<00:00,  8.52it/s]


Accuracy on the evaluation dataset: 99.0048 %
Evaluating on dataset with 1280 samples and classes: ['airplane', 'bear', 'bicycle', 'bird', 'boat', 'bottle', 'car', 'cat', 'chair', 'clock', 'dog', 'elephant', 'keyboard', 'knife', 'oven', 'truck']


Evaluating: 100%|██████████| 40/40 [00:03<00:00, 11.05it/s]


Final Results:
  Shape Accuracy: 0.1711
  Texture Accuracy: 0.5234
  Cue Accuracy: 0.6945
  Shape Bias: 0.2463
  Texture Bias: 0.7537


In [66]:
model_name = "ViT"  
model_path = "/kaggle/working/FinetunedModels/vitmodel_fine_tuned_16_classes.pth" 
eval_dataset_path1 = "/kaggle/input/labelledimagenet" 
eval_dataset_path2 = "/kaggle/input/cue-conflict" 
model = load_fine_tuned_model(model_name=model_name, model_path=model_path, num_classes=16)
evaluate_model(model, dataset_path=eval_dataset_path1, batch_size=32)
evaluate_cue_conflict_dataset(model, eval_dataset_path2, batch_size=32)

<ipython-input-50-1ecbffe9c678>:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=device))


ViT model loaded and ready for evaluation.
Evaluating on dataset with 7536 samples and classes: ['airplane', 'bear', 'bicycle', 'bird', 'boat', 'bottle', 'car', 'cat', 'chair', 'clock', 'dog', 'elephant', 'keyboard', 'knife', 'oven', 'truck']


Evaluating: 100%|██████████| 236/236 [00:46<00:00,  5.12it/s]


Accuracy on the evaluation dataset: 89.9151 %
Evaluating on dataset with 1280 samples and classes: ['airplane', 'bear', 'bicycle', 'bird', 'boat', 'bottle', 'car', 'cat', 'chair', 'clock', 'dog', 'elephant', 'keyboard', 'knife', 'oven', 'truck']


Evaluating: 100%|██████████| 40/40 [00:08<00:00,  4.96it/s]


Final Results:
  Shape Accuracy: 0.2313
  Texture Accuracy: 0.2437
  Cue Accuracy: 0.4750
  Shape Bias: 0.4868
  Texture Bias: 0.5132


In [67]:
model_name = "VGG16"  
model_path = "/kaggle/working/FinetunedModels/vgg16model_fine_tuned_16_classes.pth" 
eval_dataset_path1 = "/kaggle/input/labelledimagenet" 
eval_dataset_path2 = "/kaggle/input/cue-conflict" 
model = load_fine_tuned_model(model_name=model_name, model_path=model_path, num_classes=16)
evaluate_model(model, dataset_path=eval_dataset_path1, batch_size=32)
evaluate_cue_conflict_dataset(model, eval_dataset_path2, batch_size=32)

<ipython-input-50-1ecbffe9c678>:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=device))


VGG16 model loaded and ready for evaluation.
Evaluating on dataset with 7536 samples and classes: ['airplane', 'bear', 'bicycle', 'bird', 'boat', 'bottle', 'car', 'cat', 'chair', 'clock', 'dog', 'elephant', 'keyboard', 'knife', 'oven', 'truck']


Evaluating: 100%|██████████| 236/236 [00:28<00:00,  8.34it/s]


Accuracy on the evaluation dataset: 99.3365 %
Evaluating on dataset with 1280 samples and classes: ['airplane', 'bear', 'bicycle', 'bird', 'boat', 'bottle', 'car', 'cat', 'chair', 'clock', 'dog', 'elephant', 'keyboard', 'knife', 'oven', 'truck']


Evaluating: 100%|██████████| 40/40 [00:03<00:00, 10.05it/s]


Final Results:
  Shape Accuracy: 0.1578
  Texture Accuracy: 0.6133
  Cue Accuracy: 0.7711
  Shape Bias: 0.2047
  Texture Bias: 0.7953


In [68]:
model_name = "ResNet50"  
model_path = "/kaggle/working/FinetunedModels/resnet50model_fine_tuned_16_classes.pth" 
eval_dataset_path1 = "/kaggle/input/labelledimagenet" 
eval_dataset_path2 = "/kaggle/input/cue-conflict" 
model = load_fine_tuned_model(model_name=model_name, model_path=model_path, num_classes=16)
evaluate_model(model, dataset_path=eval_dataset_path1, batch_size=32)
evaluate_cue_conflict_dataset(model, eval_dataset_path2, batch_size=32)

<ipython-input-50-1ecbffe9c678>:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=device))


ResNet50 model loaded and ready for evaluation.
Evaluating on dataset with 7536 samples and classes: ['airplane', 'bear', 'bicycle', 'bird', 'boat', 'bottle', 'car', 'cat', 'chair', 'clock', 'dog', 'elephant', 'keyboard', 'knife', 'oven', 'truck']


Evaluating: 100%|██████████| 236/236 [00:27<00:00,  8.65it/s]


Accuracy on the evaluation dataset: 86.5048 %
Evaluating on dataset with 1280 samples and classes: ['airplane', 'bear', 'bicycle', 'bird', 'boat', 'bottle', 'car', 'cat', 'chair', 'clock', 'dog', 'elephant', 'keyboard', 'knife', 'oven', 'truck']


Evaluating: 100%|██████████| 40/40 [00:03<00:00, 11.15it/s]


Final Results:
  Shape Accuracy: 0.0953
  Texture Accuracy: 0.2359
  Cue Accuracy: 0.3312
  Shape Bias: 0.2877
  Texture Bias: 0.7123


In [51]:
model_name = "DenseNet121"  
model_path = "/kaggle/working/FinetunedModels/densenet121model_fine_tuned_16_classes.pth" 
eval_dataset_path1 = "/kaggle/input/labelledimagenet" 
eval_dataset_path2 = "/kaggle/input/cue-conflict" 
model = load_fine_tuned_model(model_name=model_name, model_path=model_path, num_classes=16)
evaluate_model(model, dataset_path=eval_dataset_path1, batch_size=32)
evaluate_cue_conflict_dataset(model, eval_dataset_path2, batch_size=32)

<ipython-input-50-1ecbffe9c678>:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=device))


DenseNet121 model loaded and ready for evaluation.
Evaluating on dataset with 1280 samples and classes: ['airplane', 'bear', 'bicycle', 'bird', 'boat', 'bottle', 'car', 'cat', 'chair', 'clock', 'dog', 'elephant', 'keyboard', 'knife', 'oven', 'truck']


Evaluating: 100%|██████████| 40/40 [00:03<00:00, 11.31it/s]


Accuracy on the evaluation dataset: 10.3906 %
Evaluating on dataset with 1280 samples and classes: ['airplane', 'bear', 'bicycle', 'bird', 'boat', 'bottle', 'car', 'cat', 'chair', 'clock', 'dog', 'elephant', 'keyboard', 'knife', 'oven', 'truck']


Evaluating: 100%|██████████| 40/40 [00:03<00:00, 11.06it/s]


Final Results:
  Shape Accuracy: 0.1039
  Texture Accuracy: 0.1875
  Cue Accuracy: 0.2914
  Shape Bias: 0.3566
  Texture Bias: 0.6434


# Logit Matching Knowledge Distillation

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm  

def logit_matching_loss(student_logits, teacher_logits, student_temperature=1.0, teacher_temperature=1.0):
    student_logits = student_logits - student_logits.mean(dim=-1, keepdim=True)
    teacher_logits = teacher_logits - teacher_logits.mean(dim=-1, keepdim=True)
    student_logits = student_logits / student_temperature
    teacher_logits = teacher_logits / teacher_temperature
    loss = F.mse_loss(student_logits, teacher_logits)
    return loss

def distill(model_class, teacher_model_class, train_loader, val_loader, num_classes, 
            epochs=10, batch_size=32, learning_rate=0.001, 
            student_temperature=2.0, teacher_temperature=2.0, alpha=0.5):
   
    student_model = model_class(num_classes=num_classes)  # Pass class reference
    optimizer = optim.Adam(student_model.parameters(), lr=learning_rate)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    student_model = student_model.to(device)
    teacher_model = teacher_model_class(num_classes=num_classes)  # Pass class reference
    teacher_model = teacher_model.to(device)
    teacher_model.eval()
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        student_model.train()
        running_loss = 0.0

        with tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", unit="batch") as pbar:
            for images, labels in pbar:
                images, labels = images.to(device), labels.to(device)
                student_outputs = student_model(images)

                with torch.no_grad():
                    teacher_outputs = teacher_model(images)

                ce_loss = criterion(student_outputs, labels)
                distillation_loss = logit_matching_loss(student_outputs, teacher_outputs, 
                                                        student_temperature=student_temperature,
                                                        teacher_temperature=teacher_temperature)
                total_loss = (1 - alpha) * ce_loss + alpha * distillation_loss
                optimizer.zero_grad()
                total_loss.backward()
                optimizer.step()
                running_loss += total_loss.item()
                pbar.set_postfix(loss=running_loss / len(pbar))

        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss / len(train_loader)}")

        student_model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = student_model(images)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        print(f"Validation Accuracy at epoch {epoch+1}: {accuracy:.2f}%")

    return student_model

In [ ]:
dataset_path = "/kaggle/input/labelledimagenet"
batch_size = 128
train_loader, val_loader = create_data_loaders(dataset_path, batch_size)
teacher_model = VGG11(num_classes=16, pretrained=True) 
student_model_class = VGG11  

distilled_model = distill(
    model_class=student_model_class, 
    teacher_model_class=teacher_model.__class__,  
    train_loader=train_loader,
    val_loader=val_loader,
    num_classes=16,
    epochs=10,
    batch_size=32,
    learning_rate=0.001,
    student_temperature=1.0,
    teacher_temperature=1.0,
    alpha=0.5
)

In [ ]:
dataset_path = "/kaggle/input/cue-conflict"
batch_size = 128
train_loader, val_loader = create_data_loaders(dataset_path, batch_size)
teacher_model = VGG16Model(num_classes=16, pretrained=True) 
student_model_class = VGG11  

distilled_model = distill(
    model_class=student_model_class, 
    teacher_model_class=teacher_model.__class__,  
    train_loader=train_loader,
    val_loader=val_loader,
    num_classes=16,
    epochs=8,
    batch_size=32,
    learning_rate=0.001,
    student_temperature=1.0,
    teacher_temperature=1.0,
    alpha=0.5
)

In [ ]:
dataset_path = "/kaggle/input/cue-conflict"
batch_size = 128
train_loader, val_loader = create_data_loaders(dataset_path, batch_size)
teacher_model = DenseNet121Model(num_classes=16, pretrained=True) 
student_model_class = VGG11  

distilled_model = distill(
    model_class=student_model_class, 
    teacher_model_class=teacher_model.__class__,  
    train_loader=train_loader,
    val_loader=val_loader,
    num_classes=16,
    epochs=8,
    batch_size=32,
    learning_rate=0.001,
    student_temperature=1.0,
    teacher_temperature=1.0,
    alpha=0.5
)

In [ ]:
dataset_path = "/kaggle/input/cue-conflict"
batch_size = 128
train_loader, val_loader = create_data_loaders(dataset_path, batch_size)
teacher_model = ResNet50Model(num_classes=16, pretrained=True) 
student_model_class = VGG11  

distilled_model = distill(
    model_class=student_model_class, 
    teacher_model_class=teacher_model.__class__,  
    train_loader=train_loader,
    val_loader=val_loader,
    num_classes=16,
    epochs=8,
    batch_size=32,
    learning_rate=0.001,
    student_temperature=1.0,
    teacher_temperature=1.0,
    alpha=0.5
)

In [ ]:
dataset_path = "/kaggle/input/cue-conflict"
batch_size = 128
train_loader, val_loader = create_data_loaders(dataset_path, batch_size)
teacher_model = ViTModel(num_classes=16, pretrained=True) 
student_model_class = VGG11  

distilled_model = distill(
    model_class=student_model_class, 
    teacher_model_class=teacher_model.__class__,  
    train_loader=train_loader,
    val_loader=val_loader,
    num_classes=16,
    epochs=8,
    batch_size=32,
    learning_rate=0.001,
    student_temperature=1.0,
    teacher_temperature=1.0,
    alpha=0.5
)

# Multi-Teacher Equal Weighted Logit Matching

In [ ]:
import torch
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm  

def logit_matching_loss(student_logits, teacher_logits_list, student_temperature=1.0, teacher_temperature=1.0):
    averaged_teacher_logits = torch.mean(torch.stack(teacher_logits_list), dim=0)
    student_logits = student_logits - student_logits.mean(dim=-1, keepdim=True)
    averaged_teacher_logits = averaged_teacher_logits - averaged_teacher_logits.mean(dim=-1, keepdim=True)
    student_logits = student_logits / student_temperature
    averaged_teacher_logits = averaged_teacher_logits / teacher_temperature
    loss = F.mse_loss(student_logits, averaged_teacher_logits)
    return loss

def multi_teacher_distill(model_class, teacher_models, train_loader, val_loader, num_classes, 
                          epochs=10, batch_size=32, learning_rate=0.001, 
                          student_temperature=2.0, teacher_temperature=2.0, alpha=0.5):
    student_model = model_class(num_classes=num_classes)
    optimizer = optim.Adam(student_model.parameters(), lr=learning_rate)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    student_model.to(device)

    for teacher_model in teacher_models:
        teacher_model.to(device)
        teacher_model.eval()
    criterion = torch.nn.CrossEntropyLoss()
    for epoch in range(epochs):
        student_model.train()
        running_loss = 0.0
        
        epoch_progress = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", ncols=100, leave=False)

        for images, labels in epoch_progress:
            images, labels = images.to(device), labels.to(device)
            student_outputs = student_model(images)
            
            teacher_logits_list = []
            for teacher_model in teacher_models:
                with torch.no_grad():
                    teacher_logits_list.append(teacher_model(images))
            
            ce_loss = criterion(student_outputs, labels) 
            distillation_loss = logit_matching_loss(student_outputs, teacher_logits_list, 
                                                     student_temperature=student_temperature,
                                                     teacher_temperature=teacher_temperature)
            total_loss = (1 - alpha) * ce_loss + alpha * distillation_loss
            optimizer.zero_grad()
            total_loss.backward()
            optimizer.step()
            running_loss += total_loss.item()
            epoch_progress.set_postfix(loss=running_loss / len(epoch_progress))

        print(f"Epoch {epoch+1}/{epochs}, Training Loss: {running_loss / len(train_loader):.4f}")
        student_model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = student_model(images)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        accuracy = 100 * correct / total
        print(f"Epoch {epoch+1}/{epochs}, Validation Accuracy: {accuracy:.2f}%")
    
    return student_model


In [ ]:
 dataset_path = "/kaggle/input/cue-conflict"
batch_size = 128
train_loader, val_loader = create_data_loaders(dataset_path, batch_size)
teacher_model_1 = VGG16Model(num_classes=16, pretrained=True) 
teacher_model_2 = ViTModel(num_classes=16, pretrained=True) 
student_model_class = VGG11  

multi_teacher_distill(
    model_class=student_model_class, 
    teacher_models=[teacher_model_1, teacher_model_2],  # List of teacher models
    train_loader=train_loader, 
    val_loader=val_loader, 
    num_classes=16, 
    epochs=8, 
    batch_size=32, 
    learning_rate=0.001, 
    student_temperature=1.0, 
    teacher_temperature=1.0, 
    alpha=0.5
)

In [ ]:
dataset_path = "/kaggle/input/cue-conflict"
batch_size = 128
train_loader, val_loader = create_data_loaders(dataset_path, batch_size)
teacher_model_1 = VGG16Model(num_classes=16, pretrained=True) 
teacher_model_2 = ViTModel(num_classes=16, pretrained=True) 
teacher_model_3 = ResNet50Model(num_classes=16, pretrained=True) 

student_model_class = VGG11  

multi_teacher_distill(
    model_class=student_model_class, 
    teacher_models=[teacher_model_1, teacher_model_2, teacher_model_3],  # List of teacher models
    train_loader=train_loader, 
    val_loader=val_loader, 
    num_classes=16, 
    epochs=8, 
    batch_size=32, 
    learning_rate=0.001, 
    student_temperature=1.0, 
    teacher_temperature=1.0, 
    alpha=0.5
)